In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# !wget ftp://ftp.cdc.noaa.gov/Datasets/cpcsoil/soilw.mon.mean.v2.nc

--2021-08-05 15:58:23--  ftp://ftp.cdc.noaa.gov/Datasets/cpcsoil/soilw.mon.mean.v2.nc
           => ‘soilw.mon.mean.v2.nc’
Resolving ftp.cdc.noaa.gov (ftp.cdc.noaa.gov)... 140.172.38.117
Connecting to ftp.cdc.noaa.gov (ftp.cdc.noaa.gov)|140.172.38.117|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Datasets/cpcsoil ... done.
==> SIZE soilw.mon.mean.v2.nc ... 235871847
==> PASV ... done.    ==> RETR soilw.mon.mean.v2.nc ... done.
Length: 235871847 (225M) (unauthoritative)

soilw.mon.mean.v2.n 100%[===================>] 224.94M  7.25MB/s    in 27s     

2021-08-05 15:58:53 (8.40 MB/s) - ‘soilw.mon.mean.v2.nc’ saved [235871847]



In [8]:
# !pip install netcdf4 plotly seaborn

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import IntProgress, HTML, VBox

In [9]:
Adults = pd.read_csv("Adults.csv")
Swarms  = pd.read_csv("Swarms.csv")
Hoppers = pd.read_csv("Hoppers.csv")
Ecology = pd.read_csv("Ecology.csv")
Control_Operations = pd.read_csv("Control_Operations.csv")

In [10]:
ecology_columns = ['NATVEGCAT','NATVEGDEN','CULTVEGCAT','SOILMOIST','IRRIGATION','MORSPECIES']
for col in ecology_columns:
    print(f'\n{col}')
    print(Ecology[col].value_counts())


NATVEGCAT
Green       227988
Drying       86833
Dry          70506
Greening     36238
Unknown      11520
Name: NATVEGCAT, dtype: int64

NATVEGDEN
Moderate    202579
Sparse      142929
Dense        66730
Unknown      20847
Name: NATVEGDEN, dtype: int64

CULTVEGCAT
Unknown     431288
Green         1342
Dry            218
Greening       145
Drying          92
Name: CULTVEGCAT, dtype: int64

SOILMOIST
Dry        209531
Wet        184116
Unknown     39438
Name: SOILMOIST, dtype: int64

IRRIGATION
Unknown    429531
No           3453
Yes           101
Name: IRRIGATION, dtype: int64

MORSPECIES
Unknown    417223
No           9223
Yes          6639
Name: MORSPECIES, dtype: int64


In [11]:
fao_data = pd.concat([Ecology, Swarms, Hoppers, Adults])

In [12]:
fao_data[["X", "Y"]].describe()

,X,Y
count,647397.000000,647397.000000
mean,28.684101,20.886411
std,30.821966,6.212712
min,-77.833333,-13.425000
25%,-3.865278,16.967000
50%,37.192778,20.055000
75%,49.688333,26.063333
max,85.950000,178.983889


In [15]:
fao_data[fao_data["Y"] > 90]#["CAT"]

,X,Y,OBJECTID,STARTDATE,TmSTARTDAT,EXACTDATE,PARTMONTH,LOCNAME,AREAHA,LOCRELIAB,...,CTLQTY,CTLQTYU,CTLARTREA,CTLARTREAU,CTLAPPHAND,CTLAPPVEHI,CTLAPPAIR,CTLAPPMECH,CTLAPPUNK,CTLESTKILL
15213,-15.157222,178.983889,15215,1999/11/14 00:00:00+00,00:00,Yes,NaN,,2.0,Exact,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25247,43.004167,153.932500,25249,2002/12/13 00:00:00+00,00:00,Yes,NaN,,20.0,Exact,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Ecology[Ecology["Y"] < 90]#["CAT"]

In [5]:
#convert date format from onbject to pandas datetime
date_columns = ['STARTDATE', 'FINISHDATE', 'CTLSTDATE', 'CTLFNDATE']
fao_data[date_columns] = fao_data[date_columns].apply(pd.to_datetime)

In [6]:
#Ensure that both Ad and Adult are categorized under the same category
fao_data.loc[fao_data['CAT'] == 'Ad', 'CAT'] = 'Adult'

In [7]:
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [8]:
before = len(fao_data)
fao_data = fao_data[fao_data["LOCRELIAB"] == "Exact"]

print(f"Dropped {before - len(fao_data)} rows with non-exact location. Remaining {len(fao_data)} rows")

Dropped 4398 rows with non-exact location. Remaining 642999 rows


In [9]:
lat_increment, lon_increment = (0.5, 0.5)

lat_to_bucket_id_fao = lambda x: int((x+90)/lat_increment)
lon_to_bucket_id_fao = lambda x: int((x+180)/lon_increment)
lat_to_bucket_id_cpc = lambda x: int((x+90)/lat_increment)
lon_to_bucket_id_cpc = lambda x: int((x)/lon_increment)

bucket_id_to_lat_fao = lambda x: (x*lat_increment) - 90
bucket_id_to_lon_fao = lambda x: (x*lon_increment) - 180
bucket_id_to_lat_cpc = lambda x: (x*lat_increment) - 90
bucket_id_to_lon_cpc = lambda x: (x*lon_increment)

In [10]:
#Replacing specific lat and lon of each row of data with its corresponding bucket_id

discretize_time  = lambda x: datetime.date(year=x.year, month=x.month, day=1)

truncated_data = fao_data.copy()
truncated_data['lat_bucket_id'] = fao_data['Y'].apply(lat_to_bucket_id_fao)
truncated_data['lon_bucket_id'] = fao_data['X'].apply(lon_to_bucket_id_fao)

truncated_data['date'] = fao_data['STARTDATE'].apply(discretize_time).astype('datetime64[ns]')
truncated_data["year"] = fao_data['STARTDATE'].dt.year
truncated_data["month"] = fao_data['STARTDATE'].dt.month_name()
# truncated_data["BREEDING"] = truncated_data["BREEDING"].fillna(0)

In [12]:
aggregate_data = truncated_data.groupby(['date','lat_bucket_id','lon_bucket_id']).agg(
# meta data
    TIME = ('STARTDATE', lambda col: col.mean()), 

# data count
    Count      = ('CAT', len),
    Swarm      = ('CAT', lambda col: sum(col=='Swarm')),
    Adult      = ('CAT', lambda col: sum(col=='Adult')),
    Hopper     = ('CAT', lambda col: sum(col=='Hopper')),
    Ecology    = ('CAT', lambda col: sum(col=='Ecology')),

# Ecology data count
    # NATVEGCAT___Ecology
    NATVEGCAT_Green    = ('NATVEGCAT', lambda col: sum(col=='Green')), 
    NATVEGCAT_Drying   = ('NATVEGCAT', lambda col: sum(col=='Drying')), 
    NATVEGCAT_Dry      = ('NATVEGCAT', lambda col: sum(col=='Dry')), 
    NATVEGCAT_Greening = ('NATVEGCAT', lambda col: sum(col=='Greening')), 

    # NATVEGDEN___Ecology
    NATVEGDEN_Moderate = ('NATVEGDEN', lambda col: sum(col=='Moderate')),
    NATVEGDEN_Sparse   = ('NATVEGDEN', lambda col: sum(col=='Sparse')), 
    NATVEGDEN_Dense    = ('NATVEGDEN', lambda col: sum(col=='Dense')), 

    # SOILMOIST___Ecology
    SOILMOIST_Dry      = ('SOILMOIST', lambda col: sum(col=='Dry')),
    SOILMOIST_Wet      = ('SOILMOIST', lambda col: sum(col=='Wet')),

    # BREEDING
    BREEDING_yes = ("BREEDING", lambda col: sum(col==1)),
    BREEDING_no = ("BREEDING", lambda col: sum(col==2)),
    BREEDING_na = ("BREEDING", lambda col: sum(col==-1)),
)

In [17]:
#Add month and day data
aggregate_data = aggregate_data.reset_index()
aggregate_data['month'] = aggregate_data['TIME'].dt.month_name()
aggregate_data['day']   = aggregate_data['TIME'].dt.day
aggregate_data['year']   = aggregate_data['TIME'].dt.year
aggregate_data["lat"] = aggregate_data["lat_bucket_id"].apply(bucket_id_to_lat_fao)
aggregate_data["lon"] = aggregate_data["lon_bucket_id"].apply(bucket_id_to_lon_fao)

available_years = aggregate_data["year"].unique().tolist()

In [17]:
def merge_soil_moisture(data, year):
    merge_on = ['lat_bucket_id', 'lon_bucket_id', 'month', 'year']
    cpc_data = pd.read_csv(f"CPC Soil Moisture/cpc_soil_moisture_{year}.csv")
    cpc_data["lat_bucket_id"] = cpc_data["lat"].apply(lat_to_bucket_id_cpc)
    cpc_data["lon_bucket_id"] = cpc_data["lon"].apply(lon_to_bucket_id_cpc)
    cpc_data["time"] = cpc_data["time"].apply(pd.to_datetime)
    cpc_data["month"] = cpc_data["time"].dt.month_name()
    data = pd.merge(data, cpc_data[merge_on + ['soilw']], how="left", on=merge_on, validate="one_to_one")
    return data

In [18]:
hover_data = ['Swarm', 
'Adult', 
'Hopper', 
"NATVEGCAT_Green",
"NATVEGCAT_Drying",
"NATVEGCAT_Dry",
"NATVEGCAT_Greening",
"SOILMOIST_Dry",
"SOILMOIST_Wet",
"soilw"
"BREEDING"
]

plot_df = aggregate_data.reset_index()

chunks = []

for year in available_years:
    chunks.append(merge_soil_moisture(plot_df[plot_df["year"] == year], year))

In [19]:
len(chunks)

37

In [22]:
fao_data_with_soilw2 = pd.merge(aggregate_data, soilw_data, how="left", on=['lat_bucket_id', 'lon_bucket_id', 'month', 'year'], validate="one_to_one")
fao_data_with_soilw2.to_csv("fao_data_with_soilw2.csv")

In [19]:
fao_data_with_soilw = pd.read_csv("fao_data_with_soilw.csv")
soilw_data = fao_data_with_soilw[['lat_bucket_id', 'lon_bucket_id', 'month', 'year', 'soilw']]

In [20]:
soilw_data

,lat_bucket_id,lon_bucket_id,month,year,soilw
0,199,453,January,1985,NaN
1,199,454,January,1985,NaN
2,200,450,January,1985,NaN
3,200,452,January,1985,NaN
4,200,457,January,1985,NaN
...,...,...,...,...,...
91123,237,509,July,2021,NaN
91124,237,510,July,2021,NaN
91125,238,506,July,2021,NaN
91126,238,507,July,2021,NaN


In [21]:
pd.concat(chunks).to_csv("fao_data_with_soilw.csv")

In [27]:
def display_map(month, year):
    data_to_plot = plot_df[(plot_df["month"] == month) & (plot_df["year"] == year)]
    data_to_plot = merge_soil_moisture(data_to_plot, year)
    fig = px.scatter_mapbox(lat='lat', lon='lon',
                          size  = 2 * (data_to_plot["month"] == month),
                          data_frame = data_to_plot,
                          hover_data=hover_data,
                          size_max=15, zoom=2
                          )
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

month = widgets.Dropdown(
    options=month_name,
    value='January',
    description='Month',
    disabled=False,
)
year = widgets.Dropdown(
    options=available_years,
    value=available_years[0],
    description='Year',
    disabled=False,
)


ui = widgets.HBox([month, year])
out = widgets.interactive_output(display_map, {"month":month, "year":year})

display(ui, out)

Output()

## Reading Soil Moisture Dataset

In [1]:
import numpy as np
import pandas as pd
import netCDF4 as nc
from datetime import date, timedelta

def get_date(days):
    start = date(1800,1,1)
    delta = timedelta(days)     
    new_date = start + delta
    return pd.to_datetime(new_date)

def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = int(n / arrays[0].size)
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [2]:
soil_moisture_ds = nc.Dataset("soilw.mon.mean.v2.nc")

In [3]:
soil_moisture_ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    Conventions: CF-1.0
    title: CPC Soil Moisture
    institution: NOAA/ESRL PSD
    dataset_title: CPC Soil Moisture
    history: Wed Oct 18 15:13:37 2017: ncks -d time,,-2 soilw.mon.mean.x.nc soilw.mon.mean.xx.nc
Wed Oct 18 15:12:08 2017: ncks -d time,,-3 soilw.mon.mean.nc soilw.mon.mean.x.nc
CPC Soil Moisture
 Obtained on Nov 2004 from CPC's website
 and written to netCDF by Cathy Smith 12/2004.
 he CPC Global monthly soil moisture dataset is a 1/2 degree resolution grid from 1948 to the present.
 The file is written in COARDS and CF compliant netCDF at NOAA ESRL/PSD https://www.esrl.noaa.gov/psd/ 
 Converted to chunked, deflated non-packed NetCDF4 Jul 2014
    NCO: 4.6.9
    References: https://www.psl.noaa.gov/data/gridded/data.cpcsoil.html
    dimensions(sizes): lat(360), lon(720), time(882)
    variables(dimensions): float32 lat(lat), float32 lon(lon), float32 soilw(time, lat, lon), 

In [30]:
for dim in soil_moisture_ds.dimensions.values():
    print(dim)

<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 360
<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 720
<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 882


In [31]:
soil_moisture_ds["lat"]

<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    long_name: Latitude
    units: degrees_north
    actual_range: [ 89.75 -89.75]
    standard_name: latitude
    axis: Y
    coordinate_defines: point
unlimited dimensions: 
current shape = (360,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [32]:
soil_moisture_ds["lon"]

<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    long_name: Longitude
    units: degrees_east
    actual_range: [2.5000e-01 3.5975e+02]
    standard_name: longitude
    axis: X
    coordinate_defines: point
unlimited dimensions: 
current shape = (720,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [33]:
soil_moisture_ds["lat"][:]

masked_array(data=[ 89.75,  89.25,  88.75,  88.25,  87.75,  87.25,  86.75,
                    86.25,  85.75,  85.25,  84.75,  84.25,  83.75,  83.25,
                    82.75,  82.25,  81.75,  81.25,  80.75,  80.25,  79.75,
                    79.25,  78.75,  78.25,  77.75,  77.25,  76.75,  76.25,
                    75.75,  75.25,  74.75,  74.25,  73.75,  73.25,  72.75,
                    72.25,  71.75,  71.25,  70.75,  70.25,  69.75,  69.25,
                    68.75,  68.25,  67.75,  67.25,  66.75,  66.25,  65.75,
                    65.25,  64.75,  64.25,  63.75,  63.25,  62.75,  62.25,
                    61.75,  61.25,  60.75,  60.25,  59.75,  59.25,  58.75,
                    58.25,  57.75,  57.25,  56.75,  56.25,  55.75,  55.25,
                    54.75,  54.25,  53.75,  53.25,  52.75,  52.25,  51.75,
                    51.25,  50.75,  50.25,  49.75,  49.25,  48.75,  48.25,
                    47.75,  47.25,  46.75,  46.25,  45.75,  45.25,  44.75,
                    44.25

In [34]:
soil_moisture_ds["time"][:].filled().shape

(882,)

In [35]:
soil_moisture_ds["time"]

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: Time
    units: days since 1800-01-01 00:00:0.0
    delta_t: 0000-01-00 00:00:00
    avg_period: 0000-01-00 00:00:00
    standard_name: time
    axis: T
    bounds: time_bnds
    coordinate_defines: start
    prev_avg_period: 0000-00-01 00:00:00
    actual_range: [54055. 80870.]
unlimited dimensions: time
current shape = (882,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [36]:
soil_moisture_ds["soilw"]

<class 'netCDF4._netCDF4.Variable'>
float32 soilw(time, lat, lon)
    long_name: Model-Calculated Monthly Mean Soil Moisture
    missing_value: -9.96921e+36
    units: mm
    valid_range: [   0. 1000.]
    dataset: CPC Monthly Soil Moisture
    var_desc: Soil Moisture
    level_desc: Surface
    statistic: Monthly Mean
    parent_stat: Other
    standard_name: lwe_thickness_of_soil_moisture_content
    cell_methods: time: mean (monthly from values)
    actual_range: [0.000000e+00 9.999318e+29]
unlimited dimensions: time
current shape = (882, 360, 720)
filling on, default _FillValue of 9.969209968386869e+36 used

In [4]:
time_lat_lon = cartesian([soil_moisture_ds["time"][:].filled(), soil_moisture_ds["lat"][:].filled(), soil_moisture_ds["lon"][:].filled()])
cpc_soil_moisture = pd.DataFrame({
    "time": time_lat_lon[:, 0],
    "lat": time_lat_lon[:, 1],
    "lon": time_lat_lon[:, 2],
    "soilw": soil_moisture_ds["soilw"][:].reshape(-1)
})

cpc_soil_moisture["time"] = cpc_soil_moisture["time"].apply(get_date)
cpc_soil_moisture = cpc_soil_moisture[cpc_soil_moisture["soilw"].notna()]
# cpc_soil_moisture.to_csv("cpc_soil_moisture.csv")

KeyboardInterrupt: 

In [ ]:
cpc_soil_moisture

In [44]:
# Save csv file for each year into "CPC Soil Moisture" folder

cpc_soil_moisture["year"] = cpc_soil_moisture["time"].dt.year
for year in cpc_soil_moisture["year"].unique():
    data_for_year = cpc_soil_moisture[cpc_soil_moisture["year"] == year]
    data_for_year.to_csv(f"CPC Soil Moisture/cpc_soil_moisture_{year}.csv")

In [4]:
cpc_soil_moisture_1985 = pd.read_csv("CPC Soil Moisture/cpc_soil_moisture_1985.csv")
cpc_soil_moisture_1985["lat_bucket_id"] = cpc_soil_moisture_1985["lat"].apply(lat_to_bucket_id_cpc)
cpc_soil_moisture_1985["lon_bucket_id"] = cpc_soil_moisture_1985["lon"].apply(lon_to_bucket_id_cpc)
cpc_soil_moisture_1985["time"] = cpc_soil_moisture_1985["time"].apply(pd.to_datetime)
cpc_soil_moisture_1985["month"] = cpc_soil_moisture_1985["time"].dt.month_name()

In [28]:
fao_1985 = aggregate_data[aggregate_data["YEAR"] == 1985].reset_index().rename(columns={"NAMED MONTH":"month", "YEAR":"year"})
merge_on = ['lat_bucket_id', 'lon_bucket_id', 'month', 'year']
fao_1985_with_soilw = pd.merge(fao_1985, cpc_soil_moisture_1985[['lat_bucket_id', 'lon_bucket_id', 'month', 'year', 'soilw']], how="left", on=merge_on, validate="one_to_one")

In [29]:
def merge_soil_moisture(data, year):
    merge_on = ['lat_bucket_id', 'lon_bucket_id', 'month', 'year']
    cpc_data = pd.read_csv(f"CPC Soil Moisture/cpc_soil_moisture_{year}.csv")
    cpc_data["lat_bucket_id"] = cpc_data["lat"].apply(lat_to_bucket_id_cpc)
    cpc_data["lon_bucket_id"] = cpc_data["lon"].apply(lon_to_bucket_id_cpc)
    cpc_data["time"] = cpc_data["time"].apply(pd.to_datetime)
    cpc_data["month"] = cpc_data["time"].dt.month_name()
    data = pd.merge(data, cpc_data[merge_on + ['soilw']], how="left", on=merge_on, validate="one_to_one")
    return data

# Reading and Preprocessing ISRIC Data

In [25]:
# read remote vrt dataset into a local tif file

from pathlib import Path
import rasterio

# the destination file 
data = Path.cwd() / 'data'
data.mkdir(exist_ok=True, parents=True)
dst = str(data/'clay_0-5cm_mean.tif')

# create the downloading url in 3 steps
sg_layer = 'clay/clay_0-5cm_mean.vrt'
location = f'https://files.isric.org/soilgrids/latest/data/{sg_layer}'
sg_url = f'/vsicurl?max_retry=3&retry_delay=1&list_dir=no&url={location}'
chunks = []

with rasterio.open(sg_url) as src:
    
    kwds = src.profile
    tags = src.tags() # Add soilgrids tags with creation info.
    kwds['driver'] = 'GTiff'
    kwds['tiled'] = True
    kwds['compress'] = 'deflate' # lzw or deflate
    kwds['dtype']: 'int16' # soilgrids datatype
    kwds['nodata'] = -32768 # default nodata

    with rasterio.open(dst, 'w', **kwds) as dst:
        for ji, window in src.block_windows(1):
            dst.update_tags(**tags)
            dst.write(src.read(window=window), window=window)

In [15]:
# read and process large local tif file

import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio

def preprocess_large_tif(tif_filename):
    chunks = []
    chunk_id = 0
    with rio.Env():
        with rio.open(tif_filename) as src:
            crs = src.crs
            for ji, window in src.block_windows(1): #600k
                # create 1D coordinate arrays (coordinates of the pixel center)
                xmin, ymax = np.around(src.xy(window.col_off, window.row_off), 9)  # src.xy(0, 0)
                xmax, ymin = np.around(src.xy(window.width-1, window.height-1), 9)  # src.xy(src.width-1, src.height-1)
                x = np.linspace(xmin, xmax, window.width)
                y = np.linspace(ymax, ymin, window.height)  # max -> min so coords are top -> bottom

                # create 2D arrays
                xs, ys = np.meshgrid(x, y)
                zs = src.read(1, window=window)

                # Apply NoData mask
                mask = src.read_masks(1, window=window) > 0
                xs, ys, zs = xs[mask], ys[mask], zs[mask]

                data = {"X": pd.Series(xs.ravel()),
                        "Y": pd.Series(ys.ravel()),
                        "Z": pd.Series(zs.ravel())}

                df = pd.DataFrame(data=data)
                geometry = gpd.points_from_xy(df.X, df.Y)
                gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
                gdf = gdf.to_crs("EPSG:4326")
                gdf["lat"] = gdf.geometry.y
                gdf["lon"] = gdf.geometry.x
                if len(gdf) > 0:
                    chunks.append(gdf)
                if ((len(chunks) % 10000) == 0) and (len(chunks) > 0):
                    chunk_id += 1
                    pd.concat(chunks).reset_index().to_csv(f"clay_0_5cm_{chunk_id}.csv")
                    chunks = []
    pd.concat(chunks).reset_index().to_csv(f"clay_0_5cm_{chunk_id}.csv")

In [16]:
preprocess_large_tif('data/clay_0-5cm_mean.tif')

KeyboardInterrupt: 

In [17]:
df = pd.read_csv("clay_0_5cm_1.csv")

In [18]:
len(df[df["Z"] == 0]) / len(df)

0.0

In [ ]:
import rasterio as rio
with rio.open('data/clay_0-5cm_mean.tif') as src:
    windows = [window for ij, window in src.block_windows()]

In [ ]:
# read remote vrt dataset, preprocess into a geopandas dataframe

import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio

def preprocess_remote_vrt(vrt_dataset_path): # e.g 'clay/clay_0-5cm_mean.vrt'
    location = f'https://files.isric.org/soilgrids/latest/data/{vrt_dataset_path}'
    sg_url = f'/vsicurl?max_retry=3&retry_delay=1&list_dir=no&url={location}'
    chunks = []

    with rio.Env():
        with rasterio.open(sg_url) as src:
            crs = src.crs
            for ji, window in src.block_windows(1):
                # create 1D coordinate arrays (coordinates of the pixel center)
                xmin, ymax = np.around(src.xy(window.col_off, window.row_off), 9)  # src.xy(0, 0)
                xmax, ymin = np.around(src.xy(window.width-1, window.height-1), 9)  # src.xy(src.width-1, src.height-1)
                x = np.linspace(xmin, xmax, window.width)
                y = np.linspace(ymax, ymin, window.height)  # max -> min so coords are top -> bottom

                # create 2D arrays
                xs, ys = np.meshgrid(x, y)
                zs = src.read(1, window=window)

                # Apply NoData mask
                mask = src.read_masks(1, window=window) > 0
                xs, ys, zs = xs[mask], ys[mask], zs[mask]

                data = {"X": pd.Series(xs.ravel()),
                        "Y": pd.Series(ys.ravel()),
                        "Z": pd.Series(zs.ravel())}

                df = pd.DataFrame(data=data)
                geometry = gpd.points_from_xy(df.X, df.Y)
                gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
                gdf = gdf.to_crs("EPSG:4326")
                gdf["lat"] = gdf.geometry.y
                gdf["lon"] = gdf.geometry.x
                chunks.append(gdf)
    return gpd.concat(chunks).reset_index()

In [ ]:
import rasterio as rio
import numpy as np
import pandas as pd
import geopandas as gpd

chunks = []
with rio.Env():
    with rio.open('data/clay_0-5cm_mean.tif') as src:
        crs = src.crs
        for ji, window in src.block_windows(1):
            # create 1D coordinate arrays (coordinates of the pixel center)
            xmin, ymax = np.around(src.xy(window.col_off, window.row_off), 9)  # src.xy(0, 0)
            xmax, ymin = np.around(src.xy(window.width-1, window.height-1), 9)  # src.xy(src.width-1, src.height-1)
            x = np.linspace(xmin, xmax, window.width)
            y = np.linspace(ymax, ymin, window.height)  # max -> min so coords are top -> bottom



            # create 2D arrays
            xs, ys = np.meshgrid(x, y)
            zs = src.read(1, window=window)

            # Apply NoData mask
            mask = src.read_masks(1, window=window) > 0
            xs, ys, zs = xs[mask], ys[mask], zs[mask]

            data = {"X": pd.Series(xs.ravel()),
                    "Y": pd.Series(ys.ravel()),
                    "Z": pd.Series(zs.ravel())}

            df = pd.DataFrame(data=data)
            geometry = gpd.points_from_xy(df.X, df.Y)
            gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
            gdf = gdf.to_crs("EPSG:4326")
            gdf["lat"] = gdf.geometry.y
            gdf["lon"] = gdf.geometry.x
            chunks.append(gdf)
#             if len(chunks) == 100:
#                 break

In [50]:
pd.concat(chunks)

,X,Y,Z,geometry,lat,lon


In [ ]:
def add_coordinate_buckets(data, lat_increment=0.5, lon_increment=0.5):
    lat_to_bucket_id = lambda x: int((x+90)/lat_increment)
    lon_to_bucket_id = lambda x: int((x+180)/lon_increment)

    bucket_id_to_lat = lambda x: (x*lat_increment) - 90
    bucket_id_to_lon = lambda x: (x*lon_increment) - 180

    data['lat_bucket_id'] = data['Y'].apply(lat_to_bucket_id)
    data['lon_bucket_id'] = data['X'].apply(lon_to_bucket_id)
    return data

# Reading and Preprocessing SMELLS dataset

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import glob
import os

def preprocess_smells_dataset(dataset_root_dir):
        chunks = []
        files = glob.glob(os.path.join(dataset_root_dir, "*", "*.tif"))
        for file in files:
                time = os.path.split(file)[-1].split(".")[0].split("_")[-1]
                year = int(time[:4])
                month = int(time[4:6])
                dekad = int(time[-1])

                with rio.Env():
                        with rio.open(file) as src:
                                crs = src.crs

                                # create 1D coordinate arrays (coordinates of the pixel center)
                                xmin, ymax = np.around(src.xy(0.00, 0.00), 4)  # src.xy(0, 0)
                                xmax, ymin = np.around(src.xy(src.height-1, src.width-1), 4)  # src.xy(src.width-1, src.height-1)
                                x = np.linspace(xmin, xmax, src.width)
                                y = np.linspace(ymax, ymin, src.height)  # max -> min so coords are top -> bottom



                                # create 2D arrays
                                xs, ys = np.meshgrid(x, y)
                                zs = src.read(1)

                                # Apply NoData mask
                                mask = src.read_masks(1) > 0
                                xs, ys, zs = xs[mask], ys[mask], zs[mask]

                        data = {"X": pd.Series(xs.ravel()),
                                "Y": pd.Series(ys.ravel()),
                                "Z": pd.Series(zs.ravel()),
                                }

                        df = pd.DataFrame(data=data)
                        df["year"] = year
                        df["month"] = month
                        df["dekad"] = dekad
                        chunks.append(df)
        return pd.concat(chunks).reset_index()

In [ ]:
smells_dataset = preprocess_smells_dataset("data/SMELLS Dataset")